# 3D Droplet Oscillation (Part 2, Postprocessing)

Results published: hopefully at some point!

This is part of the BoSSS-long-term validation test suite, which consists of several computationally expensive test-cases (runtime in the order of days), which are performed on a regular basis in order to validate the physical correctness of BoSSS simulations.

### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
//BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D_FirstPeriodStudy");

Project name is set to 'OscillatingDroplet3D_FirstPeriodStudy'.
Opening existing database 'D:\local\OscillatingDroplet3D_FirstPeriodStudy'.


In [ ]:
//add database manually:
//OpenOrCreateDatabase(@"\\dc1\userspace\smuda\Databases\OscillatingDroplet3D");
OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D_FirstPeriodStudy");
//OpenOrCreateDatabase(@"\\130.83.248.207\ValidationTests\OscillatingDroplet3D_FirstPeriodStudy");

Opening existing database '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D_FirstPeriodStudy'.


## Observing sessions

In [ ]:
var workSess = databases.Pick(0).Sessions;
workSess

#0: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta05_restart1*	08/01/2022 09:47:42	1469bf34...
#1: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta05_restart1	06/15/2022 08:37:25	7a7fc474...
#2: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta03_restart1	06/15/2022 08:37:22	351be50b...
#3: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta06_restart1	06/15/2022 08:37:23	775638f4...
#4: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta06_restart2	06/15/2022 09:32:49	43a86b55...
#5: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta02_restart1*	06/15/2022 09:32:48	9077a41e...
#6: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta07_restart1*	06/15/2022 09:32:46	5fcb29d6...
#7: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta07_restart2	06/13/2022 15:40:46	ad8e2fbb...
#8: OscillatingDroplet3D_Firs

In [ ]:
//workSess.Pick(0).Delete(true)
//workSess.Pick(8).RestartedFrom

Session 1469bf34-0d28-4239-81ce-75707912d902 deleted.


In [ ]:
// var delSess = workSess.Skip(1).Take(7);
// delSess

#0: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta05_restart1*	06/13/2022 15:59:11	e935a0ca...
#1: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta06_restart2*	06/13/2022 15:40:50	238e64bb...
#2: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta02_restart1*	06/13/2022 15:40:52	677db070...
#3: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta03_restart1*	06/13/2022 15:40:52	5165ed86...
#4: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta05_restart1*	06/13/2022 15:40:53	c021ac69...
#5: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta07_restart1*	06/13/2022 15:40:52	405a2cec...
#6: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta06_restart1*	06/13/2022 15:40:46	adb2492d...


In [ ]:
// foreach(var dS in delSess) {
//     dS.Delete(true);
// }

Session e935a0ca-9a38-44d5-8276-dfd831449006 deleted.
Session 238e64bb-e0f2-4434-a6ac-77e507c3b226 deleted.
Session 677db070-fc9f-4434-9f36-143836aab2c3 deleted.
Session 5165ed86-b474-42dc-b41b-e6d5d770b8ca deleted.
Session c021ac69-0e29-4666-a57a-0bbbabd7d8fa deleted.
Session 405a2cec-10ed-46fe-b3d4-fa12bc48bdfc deleted.
Session adb2492d-1092-4332-8151-bccf559d4f63 deleted.


In [ ]:
// var studySess = wmg.Sessions.Skip(6);
// studySess

In [ ]:
//databases

In [ ]:
//var targetDB = databases.Pick(3);
//targetDB
//databases.Pick(2).Sessions.MoveAll(targetDB);
//wmg.Sessions[0].Move(targetDB);
//wmg.Sessions.Where(s => s.Name.Contains("J432k3_arm0_case2_Oh0.1_AnalytInit"))
//wmg.Sessions[0].Delete(true)
//wmg.Sessions[8].Timesteps.Count()

In [ ]:
var sess = workSess.Pick(19);
sess

OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta05*	04/08/2022 11:30:25	6f21cae2...

In [ ]:
sess.GetSessionDirectory()

\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D_FirstPeriodStudy\sessions\de146e9e-87ad-4caf-b928-7ba443a0d324

In [ ]:
sess.DeployPath

\\fdygitrunner\ValidationTests\OscillatingDroplet3D_FirstPeriodStudy-XNSE_Solver2022Apr08_113006

In [ ]:
//sess.Timesteps
sess.Timesteps.Count

105

In [ ]:
sess.Timesteps.Skip(102).Export().WithSupersampling(2).Do();

Starting export process... Data will be written to the directory: C:\Users\smuda\AppData\Local\BoSSS\plots\sessions\OscillatingDroplet3D_FirstPeriodStudy__OD3D_J686k3_wallBC_amr1_m2_Oh01_eta05__6f21cae2-f82e-4059-a56c-139aee80115f


In [ ]:
using System.IO;

# Study Overview

In [ ]:
string[] caseS = new string[] { "m2_Oh01_eta07", "m2_Oh01_eta06", "m2_Oh01_eta05", "m2_Oh01_eta03", "m3_Oh01_eta07", "m3_Oh01_eta06", "m3_Oh01_eta05", "m4_Oh01_eta07", "m4_Oh01_eta06", "m4_Oh01_eta05", "m4_Oh01_eta03", "m4_Oh01_eta02"};
string[] gridS = new string[] { "J686" };
bool[] ShouldAnaInit = new bool[] { false };
bool[] useNewton = new bool[] { false };

In [ ]:
var studySess = new List<ISessionInfo>();
foreach(string _case in caseS) {
    foreach(var sess in workSess) {
        if(sess.Name.Contains(_case)) {
            studySess.Add(sess);
            break;
        }
    }
}

In [ ]:
//var studySess = workSess.Take(12);
studySess

#0: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta07_restart2	06/13/2022 15:40:46	ad8e2fbb...
#1: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta06_restart2	06/15/2022 09:32:49	43a86b55...
#2: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta05*	04/08/2022 11:30:25	6f21cae2...
#3: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta03_restart1*	05/20/2022 16:43:34	72f1baaa...
#4: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta07*	04/08/2022 11:30:52	714bd2c2...
#5: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta06*	04/08/2022 11:31:06	06d53502...
#6: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta05*	04/08/2022 11:31:21	c6cd6b17...
#7: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta07_restart1*	06/15/2022 09:32:46	5fcb29d6...
#8: OscillatingDroplet3D_FirstPeriodStudy	OD3D_J686k3_wallBC_a

In [ ]:
//studySess.Pick(4).RestartedFrom

00000000-0000-0000-0000-000000000000

# Plotting  droplet metrics - aspect ratio L/W

In [ ]:
//string[] metricKeys = new string[] {"theta0", "theta90x", "theta90y", "volume"};

In [ ]:
Plot2Ddata[,] PlotTableDM = new Plot2Ddata[caseS.Length,1];
for(int iCol = 0; iCol < 1; iCol++) {
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
for(int iGrd = 0; iGrd < gridS.Length; iGrd++) {   
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {

    string _case = caseS[iRow];
    string _grd = gridS[iGrd];

    // if(_case == "case2" || _case == "case3")
    //     continue; 

    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_grd)
                                      && (sess.Name.Contains("_thirdOrderInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));


    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    if(SIs.IsNullOrEmpty()) 
        continue;

    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);


    Stack<ISessionInfo>  procSIs = new Stack<ISessionInfo>();
    procSIs.Push(SI);
    var currSI = SI;
    var rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    while(!rSIs.IsNullOrEmpty()) {
        var rSI = rSIs.Single();
        procSIs.Push(rSI);
        currSI = rSI;
        rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    }
    Console.WriteLine("number of processed Sessions for {0}: {1}", _case, procSIs.Count);
    

    var plot = new Plot2Ddata();

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    if (useNewton[iNew]) {
        fmt.LineColor = LineColors.Red;
        //fmt.DashType = DashTypes.Dashed;
    } else {  
        fmt.LineColor = LineColors.Blue;
        //fmt.DashType = DashTypes.Solid;
    }
    if (!ShouldAnaInit[iAna]) {
        fmt.DashType = DashTypes.Solid;
    } else {
        fmt.DashType = DashTypes.Dashed;
    }
 
    string name = _case+"_BoSSS-"+_grd;
    // if(iAMR > 0)
    //     name = name + "-amr";
    if(ShouldAnaInit[iAna])
        name = name + "-3OrdInit"; 
    if(useNewton[iNew])
        name = name + "-Newton";



    double[] time = new double[0];
    double[] arLW = new double[0];
    
    foreach(var pSI in procSIs) {

        IDictionary<string, IList<double>> tab = null;
        try {
            tab = pSI.ReadTabulatedTextFileAsDoubles("DropletMetrics.txt", '\t');
        } catch (Exception e) {
            Console.Error.WriteLine(e.Message);
            Console.WriteLine("Skipping " + pSI);
            
            // var dummyPlot = new Plot2Ddata();
            // dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
            // PlotTableDM[iRow,iCol] = dummyPlot;
            continue;
        }

        time = time.Concat(tab["time"]).ToArray();

        double[] L = tab["L"].ToArray(); 
        double[] Wx = tab["Wx"].ToArray(); 
        double[] Wy = tab["Wy"].ToArray();
        double[] LW = new double[L.Length];
        for (int i = 0; i < L.Length; i++) {
            LW[i] = 0.5 * ((L[i] / (2.0*Wx[i])) + (L[i] / (2.0*Wy[i])));
        }
        arLW = arLW.Concat(LW).ToArray();

    }
    
    List<double> arLW_lst = new List<double>();
    List<double> time_lst = new List<double>();
    double dataThreshold = 0.1;
    for(int i = 0; i < time.Length; i++) {
        if(arLW[i] > dataThreshold) {
            arLW_lst.Add(arLW[i]);
            time_lst.Add(time[i]);
        }
    }
    arLW = arLW_lst.ToArray();
    time = time_lst.ToArray();

    plot.AddDataGroup(name, time, arLW, fmt);
   
    plot.ShowLegend = true;  

    if(PlotTableDM[iRow,iCol]==null) {
        PlotTableDM[iRow,iCol] = plot;
    } else {
        PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
    }
        
    // if(iAMR == 0) {
    //     PlotTableDM[iRow,iCol] = plot;
    // } else {
    //     PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
    // }
    
}

if(PlotTableDM[iRow,iCol] != null) {
    PlotTableDM[iRow,iCol].ShowLegend = true; 
    //PlotTableDM[iRow,iCol].LegendFont = 10;
    if(iCol == 0) {
       PlotTableDM[iRow,iCol].Title = caseS[iRow];
       PlotTableDM[iRow,iCol].Xlabel = "time t";
       PlotTableDM[iRow,iCol].Ylabel = "aspect ratio L/W";
    }
}
}
}
}
}

OD3D_J686k3_wallBC_amr1_m2_Oh01_eta07_restart2
number of processed Sessions for m2_Oh01_eta07: 3
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta06_restart2
number of processed Sessions for m2_Oh01_eta06: 3
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta05
number of processed Sessions for m2_Oh01_eta05: 1
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta03_restart1
number of processed Sessions for m2_Oh01_eta03: 2
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta07
number of processed Sessions for m3_Oh01_eta07: 1
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta06
number of processed Sessions for m3_Oh01_eta06: 1
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta05
number of processed Sessions for m3_Oh01_eta05: 1
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta07_restart1
number of processed Sessions for m4_Oh01_eta07: 2
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta06_restart1
number of processed Sessions for m4_Oh01_eta06: 2
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta05_restart1
number of processed Sessions for m4_Oh01_eta05: 2
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta03_restart1
number of processed

In [ ]:
PlotTableDM

index dataGroups LogX LogY LogBaseX LogBaseY LogX2 LogY2 XrangeMin XrangeMax YrangeMin YrangeMax Y2rangeMin Y2rangeMax X2rangeMin X2rangeMax Xlabel X2label Ylabel Y2label Title .. 0 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m2_Oh01_eta07 1 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m2_Oh01_eta06 2 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m2_Oh01_eta05 3 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m2_Oh01_eta03 4 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m3_Oh01_eta07 5 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m3_Oh01_eta06 6 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m3_Oh01_eta05 7 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m4_Oh01_eta07 8 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m4_Oh01_eta06 9 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m4_Oh01_eta05 10 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m4_Oh01_eta03 11 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False <null> <null> <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m4_Oh01_eta02

## m=2 cases (eta = {0.7, 0.6, 0.5, 0.3})

In [ ]:
var pltTab1 = PlotTableDM[0,0];
PlotTableDM[1,0].dataGroups[0].Format.LineColor = LineColors.Magenta;
pltTab1 = pltTab1.Merge(PlotTableDM[1,0]);
PlotTableDM[2,0].dataGroups[0].Format.LineColor = LineColors.Green;
pltTab1 = pltTab1.Merge(PlotTableDM[2,0]);
PlotTableDM[3,0].dataGroups[0].Format.LineColor = LineColors.Orange;
pltTab1 = pltTab1.Merge(PlotTableDM[3,0]);

In [ ]:
pltTab1.XrangeMin = 0;
pltTab1.XrangeMax = 4;
var gp = pltTab1.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m2Oh01eta07BoSSS-J686 
 
 
 
 
 m2 O h01 e ta07 B oSSS-J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M455.3,57.1 L508.7,57.1 M53.9,52.6 L54.8,52.5 L55.6,53.4 L56.5,54.8 L57.4,56.8 L58.3,59.1
 L59.1,61.7 L60.0,64.6 L60.9,67.5 L61.7,70.5 L62.6,73.5 L63.5,76.4 L64.3,79.2 L65.2,81.9
 L66.1,84.6 L67.0,87.3 L67.8,89.9 L68.7,92.5 L69.6,95.0 L70.4,97.5 L71.3,100.1 L72.2,102.6
 L73.0,105.1 L73.9,107.6 L74.8,110.2 L75.7,112.7 L76.5,115.2 L77.4,117.7 L78.3,120.2 L79.1,122.7
 L80.0,125.2 L80.9,127.6 L81.7,132.6 L82.6,134.7 L83.5,138.4 L84.4,140.5 L85.2,142.7 L86.1,144.8
 L87.0,146.8 L87.8,148.7 L88.7,150.6 L89.6,152.7 L90.5,154.9 L91.3,157.1 L92.2,159.3 L93.1,161.4
 L93.9,164.1 L94.8,166.2 L95.7,168.3 L96.5,170.2 L97.4,172.0 L98.3,175.1 L99.2,176.8 L100.0,178.6
 L100.9,181.1 L101.8,182.8 L102.6,184.5 L103.5,186.2 L104.4,188.0 L105.2,189.7 L106.1,191.8 L107.0,193.7
 L107.9,195.6 L108.7,197.5 L109.6,199.2 L110.5,201.2 L111.3,203.1 L112.2,204.7 L113.1,206.6 L113.9,208.4
 L114.8,210.2 L115.7,211.9 L116.6,213.6 L117.4,215.3 L118.3,217.0 L119.2,218.6 L120.0,220.3 L120.9,221.9
 L121.8,223.6 L122.6,225.2 L123.5,226.8 L124.4,228.5 L125.3,230.1 L126.1,231.7 L127.0,233.0 L127.9,234.7
 L128.7,236.3 L129.6,238.0 L130.5,239.6 L131.4,241.2 L132.2,242.7 L133.1,244.0 L134.0,245.6 L134.8,247.3
 L135.7,248.9 L136.6,250.4 L137.4,252.0 L138.3,253.1 L139.2,254.7 L140.1,256.3 L140.9,258.0 L141.8,259.5
 L142.7,260.8 L143.5,262.3 L144.4,264.1 L145.3,265.7 L146.1,267.4 L147.0,269.0 L147.9,270.7 L148.8,272.2
 L149.6,273.8 L150.5,275.4 L151.4,276.9 L152.2,278.5 L153.1,280.1 L154.0,281.6 L154.8,283.3 L155.7,284.9
 L156.6,286.6 L157.5,288.2 L158.3,289.9 L159.2,291.5 L160.1,293.3 L160.9,294.9 L161.8,296.6 L162.7,298.3
 L163.6,300.0 L164.4,301.7 L165.3,303.5 L166.2,305.2 L167.0,307.0 L167.9,308.9 L168.8,310.7 L169.6,312.5
 L170.5,314.4 L171.4,317.3 L172.3,319.2 L173.1,320.8 L174.0,322.6 L174.9,324.4 L175.7,326.4 L176.6,328.4
 L177.5,330.4 L178.3,333.1 L179.2,335.0 L180.1,336.8 L181.0,339.1 L181.8,341.0 L182.7,342.9 L183.6,344.9
 L184.4,347.1 L185.3,349.1 L186.2,351.2 L187.0,353.3 L187.9,355.5 L188.8,357.7 L189.7,359.8 L190.5,362.0
 L191.4,364.1 L192.3,366.3 L193.1,368.6 L194.0,370.9 L194.9,373.1 L195.8,375.4 L196.6,377.2 L197.5,379.5
 L198.4,381.8 L199.2,383.8 L200.1,386.0 L201.0,388.2 L201.8,390.6 L202.7,392.9 L203.6,395.1 L204.5,397.2
 L205.3,399.5 L206.2,401.8 L207.1,404.0 L207.9,406.3 L208.8,408.6 L209.7,410.8 L210.5,413.1 L211.4,415.4
 L212.3,417.5 L213.2,419.8 L214.0,422.0 L214.9,424.3 L215.8,426.5 L216.6,428.6 L217.5,430.5 L218.4,432.7
 L219.2,434.9 L220.1,437.0 L221.0,439.2 L221.9,441.4 L222.7,443.4 L223.6,445.5 L224.5,447.5 L225.3,449.5
 L226.2,451.4 L227.1,453.4 L228.0,455.4 L228.8,457.4 L229.7,459.4 L230.6,461.3 L231.4,463.3 L232.3,465.2
 L233.2,467.1 L234.0,469.0 L234.9,470.8 L235.8,472.7 L236.7,474.5 L237.5,476.2 L238.4,478.1 L239.3,479.8
 L240.1,481.5 L241.0,483.2 L241.9,484.8 L242.7,486.4 L243.6,488.2 L244.5,489.7 L245.4,491.5 L246.2,492.9
 L247.1,494.4 L248.0,496.2 L248.8,498.1 L249.7,499.5 L250.6,501.0 L251.4,502.3 L252.3,503.3 L253.2,504.3
 L254.1,505.9 L254.9,506.8 L255.8,507.9 L256.7,509.1 L257.5,510.6 L258.4,511.6 L259.3,512.6 L260.1,513.8
 L261.0,514.9 L261.9,516.0 L262.8,517.2 L263.6,518.1 L264.5,519.1 L265.4,520.1 L266.2,521.0 L267.1,522.0
 L268.0,522.8 L268.9,523.7 L269.7,524.4 L270.6,525.3 L271.5,526.0 L272.3,526.7 L273.2,527.5 

In [ ]:
for(int i = 0; i < 4; i++) {
    Plot2Ddata dat = new Plot2Ddata();
    var grp = PlotTableDM[i,0].dataGroups[0];
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/m2/" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

## m=3 cases (eta = {0.7, 0.6, 0.5})

In [ ]:
var pltTab2 = PlotTableDM[4,0];
PlotTableDM[5,0].dataGroups[0].Format.LineColor = LineColors.Magenta;
pltTab2 = pltTab2.Merge(PlotTableDM[5,0]);
PlotTableDM[6,0].dataGroups[0].Format.LineColor = LineColors.Green;
pltTab2 = pltTab2.Merge(PlotTableDM[6,0]);

In [ ]:
pltTab1.XrangeMin = 0;
pltTab1.XrangeMax = 2;
var gp = pltTab2.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m3Oh01eta07BoSSS-J686 
 
 
 
 
 m3 O h01 e ta07 B oSSS-J686 
 
 
 
 
 
 m3Oh01eta06BoSSS-J686 
 
 
 m3 O h01 e ta06 B oSSS-J686 
 
 
 
 
 
 m3Oh01eta05BoSSS-J686 
 
 
 m3 O h01 e ta05 B oSSS-J686 
 
 
 
	<path stroke='rgb( 0, 255, 0)' d='M455.3,105.1 L508.7,105.1 M71.3,179.4 L109.0,155.0 L117.7,150.1 L164.1,131.7 L175.7,126.3 L193.1,122.6
 L196.0,121.1 L222.1,111.9 L225.0,111.2 L233.8,109.3 L251.2,105.4 L259.9,134.8 L288.9,524.4 L291.8,526.2
 L297.6,522.4 L300.5,516.1 L306.3,103.3 L309.2,119.1 L312.1,120.5 L317.9,123.1 L335.3,133.0 L346.9,135.0
 L349.8,113.5 L352.7,113.7 L355.6,114.5 L358.5,115.3 L364.3,116.8 L370.1,145.9 L375.9,147.3 L381.7,151.7
 L384.6,149.6 L393.3,151.8 L402.0,159.5 L407.8,156.2 L413.6,517.5 L419.4,130.7 L425.2,484.8 L428.1,520.1
 L433.9,182.6 L436.8,208.7 L439.7,513.5 L445.5,490.4 L448.4,135.6 L451.3,485.2 L454.2,135.9 L460.0,493.9
 L462.9,487.5 L471.6,484.4 L474.5,135.1 L477.4,490.8 L483.2,134.9 L486.1,485.2 L489.0,134.5 L491.9,152.6
 L494.8,193.1 L497.7,133.3 L500.6,133.5 L506.4,133.6 L509.3,133.7 L518.0,488.9 L526.7,178.8 L529.6,132.9
 L535.4,148.1 L541.2,147.3 L544.1,133.7 L547.0,135.0 L552.8,135.1 L555.7,176.8 L558.6,512.7 L561.5,158.3
 L564.4,167.3 L567.3,135.3 L570.2,165.3 L573.1,153.6 L576.1,135.7 L579.0,135.0 L581.9,153.9 L584.8,152.2
 L587.7,136.1 L590.6,136.3 L599.3,136.0 L602.2,136.5 L605.1,136.6 L608.0,137.3 L610.9,137.7 L613.8,161.6
 L616.7,138.7 L619.6,139.2 L622.5,139.8 L625.4,140.2 L628.3,140.8 L634.1,163.0 L637.0,171.9 L639.9,143.5
 L642.8,162.0 L645.7,144.4 L648.6,170.6 L654.4,177.0 L660.2,144.3 '/>

In [ ]:
// foreach(var grp in pltTab1.dataGroups) {
//     Plot2Ddata dat = new Plot2Ddata();
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m2/" + pltTab1.Title + "/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

## m=4 cases (eta = {0.7, 0.6, 0.5, 0.3, 0.2})

In [ ]:
var pltTab3 = PlotTableDM[7,0];
PlotTableDM[8,0].dataGroups[0].Format.LineColor = LineColors.Magenta;
pltTab3 = pltTab3.Merge(PlotTableDM[8,0]);
PlotTableDM[9,0].dataGroups[0].Format.LineColor = LineColors.Green;
pltTab3 = pltTab3.Merge(PlotTableDM[9,0]);
PlotTableDM[10,0].dataGroups[0].Format.LineColor = LineColors.Orange;
pltTab3 = pltTab3.Merge(PlotTableDM[10,0]);
PlotTableDM[11,0].dataGroups[0].Format.LineColor = LineColors.Red;
pltTab3 = pltTab3.Merge(PlotTableDM[11,0]);

In [ ]:
pltTab3.XrangeMin = 0;
pltTab3.XrangeMax = 4;
var gp = pltTab3.ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.8 
 
 
 
 
 0.9 
 
 
 
 
 1 
 
 
 
 
 1.1 
 
 
 
 
 1.2 
 
 
 
 
 1.3 
 
 
 
 
 1.4 
 
 
 
 
 1.5 
 
 
 
 
 1.6 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m4Oh01eta07BoSSS-J686 
 
 
 
 
 m4 O h01 e ta07 B oSSS-J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M455.3,57.1 L508.7,57.1 M53.9,192.1 L54.8,192.2 L55.6,192.1 L56.5,191.8 L57.4,191.5 L58.3,191.2
 L59.1,190.7 L60.0,190.2 L60.9,189.6 L61.7,189.5 L62.6,188.8 L63.5,187.8 L64.3,186.9 L65.2,185.9
 L66.1,185.6 L67.0,184.3 L67.8,183.1 L68.7,181.9 L69.6,182.0 L70.4,180.7 L71.3,179.6 L72.2,178.7
 L73.0,177.6 L73.9,176.4 L74.8,175.0 L75.7,173.4 L76.5,172.3 L77.4,171.4 L78.3,170.1 L79.1,169.0
 L80.0,167.7 L80.9,166.7 L81.7,165.0 L82.6,163.6 L83.5,162.2 L84.4,160.7 L85.2,156.0 L86.1,153.9
 L87.0,151.8 L87.8,150.3 L88.7,148.4 L89.6,144.9 L90.5,143.4 L91.3,140.8 L93.1,135.3 L93.9,131.8
 L94.8,128.7 L95.7,126.2 L96.5,123.0 L97.4,120.1 L98.3,117.1 L99.2,114.0 L100.0,111.7 L100.9,107.8
 L101.8,105.6 L102.6,102.6 L103.5,99.2 L104.4,96.7 L105.2,93.6 L106.1,91.4 L107.0,89.0 L107.9,86.6
 L108.7,83.5 L109.6,81.2 L110.5,79.4 L111.3,77.8 L116.6,72.6 L117.4,73.1 L120.0,74.0 L120.9,76.4
 L121.8,79.2 L122.6,80.3 L124.4,87.0 L125.3,93.0 L126.1,96.5 L127.0,102.6 L127.9,108.8 L128.7,115.4
 L129.6,122.2 L130.5,129.2 L131.4,136.5 L132.2,144.1 L133.1,152.2 L134.0,160.2 L134.8,169.6 L135.7,179.0
 L136.6,188.1 L137.4,197.8 L138.3,207.9 L139.2,218.2 L140.1,227.5 L140.9,234.4 L141.8,238.7 L142.7,243.5
 L143.5,248.7 L144.4,254.8 L145.3,260.0 L147.9,273.6 L148.8,278.0 L162.7,250.6 L163.6,247.1 L164.4,243.9
 L165.3,240.6 L168.8,226.3 L169.6,222.6 L171.4,215.1 L172.3,210.2 L173.1,210.6 L174.0,205.2 L181.0,189.4
 L181.8,186.8 L182.7,186.3 L183.6,185.8 L184.4,185.6 L186.2,185.3 L187.9,182.1 L188.8,182.7 L189.7,183.5
 L190.5,182.6 L191.4,183.8 L192.3,184.1 L193.1,185.5 L194.0,186.9 L194.9,188.0 L195.8,189.5 L196.6,188.8
 L197.5,190.5 L198.4,192.1 L199.2,191.9 L200.1,193.8 L201.0,192.8 L201.8,195.1 L202.7,195.8 L203.6,198.3
 L204.5,199.1 L205.3,200.8 L206.2,203.5 L207.1,205.6 L207.9,207.4 L208.8,209.2 L209.7,209.2 L210.5,211.6
 L211.4,213.8 L212.3,215.7 L213.2,216.0 L214.0,217.9 L214.9,220.4 L215.8,222.6 L216.6,224.7 L217.5,226.7
 L218.4,228.6 L219.2,229.5 L220.1,231.4 L221.0,233.5 L221.9,235.7 L222.7,237.6 L223.6,239.5 L224.5,241.3
 L225.3,243.2 L226.2,245.0 L227.1,246.8 L228.0,248.6 L228.8,250.4 L229.7,252.2 L230.6,254.1 L231.4,256.0
 L232.3,257.8 L233.2,259.7 L234.0,261.6 L234.9,263.5 L235.8,265.5 L236.7,269.0 L237.5,271.7 L238.4,273.8
 L239.3,275.7 L240.1,277.6 L241.0,279.4 L241.9,281.2 L242.7,283.1 L243.6,285.2 L244.5,287.3 L245.4,289.5
 L246.2,291.8 L247.1,294.1 L248.0,296.5 L248.8,298.9 L249.7,301.6 L250.6,304.0 L251.4,306.3 L252.3,309.0
 L253.2,311.3 L254.1,313.7 L254.9,317.5 L255.8,320.2 L256.7,322.4 L257.5,325.2 L258.4,327.6 L259.3,330.4
 L260.1,334.0 L261.0,337.0 L261.9,339.4 L262.8,342.2 L263.6,345.1 L264.5,350.1 L265.4,352.3 L267.1,359.8
 L268.0,362.7 L268.9,365.8 L269.7,369.1 L271.5,375.8 L272.3,381.1 L273.2,386.5 L274.1,391.8 L274.9,395.5
 L275.8,399.9 L276.7,404.0 L277.6,406.2 L278.4,410.1 L279.3,414.0 L280.2,418.1 L281.0,421.2 L281.9,424.7
 L282.8,428.1 L283.6,428.4 L284.5,432.6 L285.4,434.7 L286.3,438.6 L287.1,442.2 L288.0,445.6 L289.7,452.1
 L290.6,455.0 L292.3,461.5 L293.2,464.8 L294.1,468.0 L298.4,481.6 L299.3,484.0 L300.2,486.3 L301.1,488.4
 L301.9,490.5 L302.8,492.6 L303.7,496.6 L304.5,498.1 L305.4,499.8 L306.3,501.6 L307.1,503.3 L308.0,505.0
 L308.9,507.1 

In [ ]:
for(int i = 7; i < 12; i++) {
    Plot2Ddata dat = new Plot2Ddata();
    var grp = PlotTableDM[i,0].dataGroups[0];
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/m4/" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}

# Plotting energies

In [ ]:
string[] energyKeys = new string[] {"kineticEnergy", "surfaceEnergy", "totalEnergy"}; //, "kineticDissipation"};
double surfEnrgy0 = Math.PI;    // surface energy for a sphere with r = 1 and sigma = 1 on a quarter domain

In [ ]:
Plot2Ddata[,] PlotTableEnrgy = new Plot2Ddata[caseS.Length, 3];
for(int iRow = 0; iRow < PlotTableEnrgy.GetLength(0); iRow++) { // loop over cases
for(int iCol = 0; iCol < PlotTableEnrgy.GetLength(1); iCol++) {
for(int iGrd = 0; iGrd < gridS.Length; iGrd++) {
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {
   
    string _case = caseS[iRow];
    string _grd = gridS[iGrd];

    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_grd)
                                      && (sess.Name.Contains("_thirdOrderInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));
    
    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }

    if(SIs.IsNullOrEmpty()) 
    continue;
    
    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    Stack<ISessionInfo>  procSIs = new Stack<ISessionInfo>();
    procSIs.Push(SI);
    var currSI = SI;
    var rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    while(!rSIs.IsNullOrEmpty()) {
        var rSI = rSIs.Single();
        procSIs.Push(rSI);
        currSI = rSI;
        rSIs = workSess.Where(sess => sess.ID.Equals(currSI.RestartedFrom));
    }

    var plot = new Plot2Ddata();

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    //fmt.LineColor = LineColors.Blue;
    if (ShouldAnaInit[iAna]) 
        fmt.DashType = DashTypes.Dashed;
    else  
        fmt.DashType = DashTypes.Solid;   
    var fmtSE = fmt.CloneAs();   
    fmtSE.LineColor = LineColors.Blue;    
    var fmtKE = fmt.CloneAs();   
    fmtKE.LineColor = LineColors.Red;    
    var fmtTE = fmt.CloneAs();   
    fmtTE.LineColor = LineColors.Blue;    

    string description = _grd;
    if(ShouldAnaInit[iAna]) 
        description = description + "-3OrdInit";
    if(useNewton[iNew]) 
        description = description + "-Newton";
    string nameSE = _case+" surfaceEnergy "+description;
    string nameKE = _case+" kineticEnergy "+description;
    string nameTE = _case+" totalEnergy "+description;


    List<double> time = new List<double>();
    List<double> procSurfEnrgy = new List<double>();
    List<double> kinEnrgy = new List<double>();
    List<double> totEnrgy = new List<double>();


    foreach(var pSI in procSIs) {

        //Console.WriteLine("processing Session: {0}", pSI.Name);
        IDictionary<string, IList<double>> tab = null;
        try {
            tab = pSI.ReadTabulatedTextFileAsDoubles("EnergyLogValues.txt", '\t');
        } catch (Exception e) {
            Console.Error.WriteLine(e.Message);
            Console.WriteLine("Skipping " + pSI);
            
            // var dummyPlot = new Plot2Ddata();
            // dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
            // PlotTableDM[iRow,iCol] = dummyPlot;
            continue;
        }
        
        time = time.Concat(tab["time"]).ToList();
        
        if(iCol == 0) {
            foreach(double sEnrgyVal in tab["surfaceEnergy"]) {
                procSurfEnrgy.Add(sEnrgyVal - surfEnrgy0);
            }  
        }
    
        if(iCol == 1) {
            kinEnrgy = kinEnrgy.Concat(tab["kineticEnergy"]).ToList();
        }

        if(iCol == 2) {
            totEnrgy = totEnrgy.Concat(tab["totalEnergy"]).ToList(); 
        }

    }

    if(iCol == 0) {
        plot.AddDataGroup(nameSE, time, procSurfEnrgy, fmtSE);
    }
    if(iCol == 1) {
        plot.AddDataGroup(nameKE, time, kinEnrgy, fmtKE);
    }
    if(iCol == 2) {
        plot.AddDataGroup(nameTE, time, totEnrgy, fmtTE);
    }
    
   
    //plot.ShowLegend = iCol == 2 && iRow == 2;
    plot.ShowLegend = true;
    
    //plot.Title = SI.Name;
        
    if(PlotTableEnrgy[iRow,iCol]==null) {
        PlotTableEnrgy[iRow,iCol] = plot;
    } else {
        PlotTableEnrgy[iRow,iCol] = PlotTableEnrgy[iRow,iCol].Merge(plot);
    }
        
    PlotTableEnrgy[iRow,iCol].XrangeMin = 0;
    PlotTableEnrgy[iRow,iCol].XrangeMax = 7;
}
}
}
}
}

OD3D_J686k3_wallBC_amr1_m2_Oh01_eta07_restart1
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta07_restart1
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta07_restart1
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta06_restart1
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta06_restart1
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta06_restart1
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta05_restart1
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta05_restart1
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta05_restart1
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta03_restart1
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta03_restart1
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta03_restart1
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta07
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta07
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta07
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta06
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta06
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta06
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta05
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta05
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta05
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta07
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta07
OD3D_J686k3_wallBC

In [ ]:
PlotTableEnrgy

index dataGroups LogX LogY LogBaseX LogBaseY LogX2 LogY2 XrangeMin XrangeMax YrangeMin YrangeMax Y2rangeMin Y2rangeMax X2rangeMin X2rangeMax Xlabel X2label Ylabel Y2label Title .. 0 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 1 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 2 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 3 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 4 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 5 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 6 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 7 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 8 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 9 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 10 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 11 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 12 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 13 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 14 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 15 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 16 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 17 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 18 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 19 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> <null> 20 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 7 <null> <null> <null> <null> <null> <null> 

## m=2 cases (eta = {0.7, 0.6, 0.5, 0.3})

In [ ]:
Plot2Ddata[] pltTabE1 = new Plot2Ddata[3];
for(int i = 0; i < 3; i++) {
    pltTabE1[i] = new Plot2Ddata();
    PlotTableEnrgy[0,i].dataGroups[0].Format.LineColor = LineColors.Blue;
    pltTabE1[i].AddDataGroup(PlotTableEnrgy[0,i].dataGroups[0]);
    PlotTableEnrgy[1,i].dataGroups[0].Format.LineColor = LineColors.Magenta;
    pltTabE1[i] = pltTabE1[i].Merge(PlotTableEnrgy[1,i]);
    PlotTableEnrgy[2,i].dataGroups[0].Format.LineColor = LineColors.Green;
    pltTabE1[i] = pltTabE1[i].Merge(PlotTableEnrgy[2,i]);
    PlotTableEnrgy[3,i].dataGroups[0].Format.LineColor = LineColors.Orange;
    pltTabE1[i] = pltTabE1[i].Merge(PlotTableEnrgy[3,i]);
}

In [ ]:
pltTabE1[0].XrangeMin = 0;
pltTabE1[0].XrangeMax = 4;
var gp = pltTabE1[0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.05 
 
 
 
 
 0.1 
 
 
 
 
 0.15 
 
 
 
 
 0.2 
 
 
 
 
 0.25 
 
 
 
 
 0.3 
 
 
 
 
 0.35 
 
 
 
 
 0.4 
 
 
 
 
 0.45 
 
 
 
 
 0.5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m2Oh01eta07 surfaceEnergy J686 
 
 
 
 
 m2 O h01 e ta07 surfaceEnergy J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M355.4,57.1 L408.8,57.1 M62.2,75.8 L63.1,75.8 L63.9,76.2 L64.8,76.9 L65.6,77.9 L66.5,79.1
 L67.4,80.4 L68.2,81.9 L69.1,83.4 L69.9,85.1 L70.8,86.8 L71.7,88.5 L72.5,90.3 L73.4,92.1
 L74.2,94.0 L75.1,95.8 L76.0,97.7 L76.8,99.7 L77.7,101.6 L78.5,103.6 L79.4,105.6 L80.3,107.5
 L81.1,109.5 L82.0,111.5 L82.8,113.6 L83.7,115.6 L84.6,117.6 L85.4,119.6 L86.3,121.7 L87.1,123.7
 L88.0,125.8 L88.9,127.8 L89.7,130.3 L90.6,132.3 L91.4,134.5 L92.3,136.6 L93.2,138.7 L94.0,140.8
 L94.9,142.9 L95.7,145.0 L96.6,147.2 L97.5,149.3 L98.3,151.5 L99.2,153.7 L100.0,155.9 L100.9,158.1
 L101.8,160.3 L102.6,162.6 L103.5,164.8 L104.3,167.1 L105.2,169.4 L106.1,171.7 L106.9,174.0 L107.8,176.4
 L108.6,178.7 L109.5,181.1 L110.4,183.4 L111.2,185.8 L112.1,188.2 L112.9,190.6 L113.8,193.1 L114.7,195.6
 L115.5,198.1 L116.4,200.5 L117.2,203.1 L118.1,205.6 L119.0,208.1 L119.8,210.7 L120.7,213.3 L121.5,215.9
 L122.4,218.5 L123.3,221.1 L124.1,223.8 L125.0,226.5 L125.8,229.1 L126.7,231.8 L127.6,234.6 L128.4,237.3
 L129.3,240.0 L130.1,242.8 L131.0,245.6 L131.8,248.4 L132.7,251.2 L133.6,254.0 L134.4,256.8 L135.3,259.7
 L136.1,262.6 L137.0,265.4 L137.9,268.3 L138.7,271.2 L139.6,274.1 L140.4,277.0 L141.3,280.0 L142.2,282.9
 L143.0,285.8 L143.9,288.8 L144.7,291.7 L145.6,294.7 L146.5,297.7 L147.3,300.6 L148.2,303.6 L149.0,306.6
 L149.9,309.6 L150.8,312.7 L151.6,315.7 L152.5,318.7 L153.3,321.8 L154.2,324.8 L155.1,327.9 L155.9,330.9
 L156.8,334.0 L157.6,337.0 L158.5,340.1 L159.4,343.1 L160.2,346.2 L161.1,349.3 L161.9,352.4 L162.8,355.5
 L163.7,358.5 L164.5,361.7 L165.4,364.8 L166.2,367.9 L167.1,370.9 L168.0,374.1 L168.8,377.2 L169.7,380.4
 L170.5,383.5 L171.4,386.7 L172.3,389.8 L173.1,393.0 L174.0,396.1 L174.8,399.3 L175.7,402.5 L176.6,405.7
 L177.4,408.8 L178.3,412.0 L179.1,415.2 L180.0,418.4 L180.9,421.6 L181.7,424.9 L182.6,428.1 L183.4,431.3
 L184.3,434.5 L185.2,437.6 L186.0,440.9 L186.9,444.1 L187.7,447.2 L188.6,450.4 L189.5,453.6 L190.3,456.8
 L191.2,460.0 L192.0,463.1 L192.9,466.3 L193.8,469.4 L194.6,472.5 L195.5,475.6 L196.3,478.7 L197.2,481.8
 L198.1,484.8 L198.9,487.8 L199.8,490.9 L200.6,493.8 L201.5,496.8 L202.4,499.7 L203.2,502.5 L204.1,505.4
 L204.9,508.2 L205.8,510.9 L206.7,513.6 L207.5,516.3 L208.4,519.0 L209.2,521.6 L210.1,524.1 L211.0,526.5
 L211.8,529.0 L212.7,531.3 L213.5,533.6 L214.4,535.8 L215.3,538.0 L216.1,540.0 L217.0,542.1 L217.8,544.0
 L218.7,545.8 L219.6,547.6 L220.4,549.3 L221.3,550.9 L222.1,552.4 L223.0,553.8 L223.9,555.0 L224.7,556.2
 L225.6,557.3 L226.4,558.4 L227.3,559.3 L228.2,560.1 L229.0,560.8 L229.9,561.4 L230.7,561.9 L231.6,562.3
 L232.5,562.6 L233.3,562.9 L234.2,563.0 L235.0,563.0 L235.9,562.9 L236.8,562.7 L237.6,562.4 L238.5,562.1
 L239.3,561.6 L240.2,561.1 L241.1,560.4 L241.9,559.7 L242.8,558.9 L243.6,558.0 L244.5,557.0 L245.4,555.9
 L246.2,554.8 L247.1,553.5 L247.9,552.2 L248.8,550.8 L249.7,549.3 L250.5,547.8 L251.4,546.2 L252.2,544.6
 L253.1,542.9 L254.0,541.1 L254.8,539.1 L255.7,537.2 L256.5,535.3 L257.4,533.2 L258.3,531.1 L259.1,529.0
 L260.0,526.9 L260.8,524.7 L261.7,522.5 L262.6,520.3 L263.4,518.0 L264.3,515.8 L265.1,513.5 L266.0,511.2
 L266.9,508.9 L267.7,506.5 L268.6,504.2 L

In [ ]:
pltTabE1[1].XrangeMin = 0;
pltTabE1[1].XrangeMax = 4;
var gp = pltTabE1[1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.05 
 
 
 
 
 0.1 
 
 
 
 
 0.15 
 
 
 
 
 0.2 
 
 
 
 
 0.25 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m2Oh01eta07 kineticEnergy J686 
 
 
 
 
 m2 O h01 e ta07 kineticEnergy J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M355.4,57.1 L408.8,57.1 M62.2,564.0 L63.1,563.7 L63.9,562.7 L64.8,561.4 L65.6,559.8 L66.5,558.0
 L67.4,556.1 L68.2,554.1 L69.1,552.0 L69.9,550.0 L70.8,547.8 L71.7,545.7 L72.5,543.5 L73.4,541.3
 L74.2,539.0 L75.1,536.7 L76.0,534.5 L76.8,532.2 L77.7,529.9 L78.5,527.5 L79.4,525.2 L80.3,522.9
 L81.1,520.5 L82.0,518.2 L82.8,515.9 L83.7,513.5 L84.6,511.1 L85.4,508.7 L86.3,506.3 L87.1,503.9
 L88.0,501.5 L88.9,499.1 L89.7,496.6 L90.6,494.1 L91.4,491.6 L92.3,489.1 L93.2,486.5 L94.0,484.0
 L94.9,481.4 L95.7,478.8 L96.6,476.2 L97.5,473.5 L98.3,470.8 L99.2,468.1 L100.0,465.4 L100.9,462.6
 L101.8,459.8 L102.6,457.0 L103.5,454.1 L104.3,451.3 L105.2,448.4 L106.1,445.4 L106.9,442.4 L107.8,439.4
 L108.6,436.4 L109.5,433.3 L110.4,430.3 L111.2,427.1 L112.1,424.0 L112.9,420.8 L113.8,417.6 L114.7,414.3
 L115.5,411.1 L116.4,407.8 L117.2,404.5 L118.1,401.1 L119.0,397.7 L119.8,394.3 L120.7,390.9 L121.5,387.4
 L122.4,384.0 L123.3,380.5 L124.1,376.9 L125.0,373.4 L125.8,369.8 L126.7,366.2 L127.6,362.6 L128.4,359.0
 L129.3,355.4 L130.1,351.7 L131.0,348.0 L131.8,344.3 L132.7,340.6 L133.6,336.9 L134.4,333.2 L135.3,329.4
 L136.1,325.7 L137.0,321.9 L137.9,318.1 L138.7,314.3 L139.6,310.5 L140.4,306.7 L141.3,302.9 L142.2,299.1
 L143.0,295.3 L143.9,291.5 L144.7,287.6 L145.6,283.8 L146.5,280.0 L147.3,276.1 L148.2,272.3 L149.0,268.5
 L149.9,264.7 L150.8,260.8 L151.6,257.0 L152.5,253.2 L153.3,249.3 L154.2,245.5 L155.1,241.7 L155.9,237.9
 L156.8,234.1 L157.6,230.3 L158.5,226.5 L159.4,222.7 L160.2,218.9 L161.1,215.1 L161.9,211.3 L162.8,207.5
 L163.7,203.7 L164.5,200.0 L165.4,196.2 L166.2,192.4 L167.1,188.6 L168.0,184.8 L168.8,181.1 L169.7,177.3
 L170.5,173.5 L171.4,169.8 L172.3,166.0 L173.1,162.3 L174.0,158.6 L174.8,154.8 L175.7,151.1 L176.6,147.4
 L177.4,143.8 L178.3,140.2 L179.1,136.5 L180.0,132.9 L180.9,129.4 L181.7,125.9 L182.6,122.4 L183.4,118.9
 L184.3,115.5 L185.2,112.2 L186.0,108.9 L186.9,105.6 L187.7,102.4 L188.6,99.3 L189.5,96.2 L190.3,93.1
 L191.2,90.2 L192.0,87.3 L192.9,84.5 L193.8,81.8 L194.6,79.2 L195.5,76.6 L196.3,74.2 L197.2,71.9
 L198.1,69.6 L198.9,67.5 L199.8,65.5 L200.6,63.5 L201.5,61.8 L202.4,60.1 L203.2,58.6 L204.1,57.1
 L204.9,55.9 L205.8,54.8 L206.7,53.8 L207.5,52.9 L208.4,52.2 L209.2,51.7 L210.1,51.3 L211.0,51.1
 L211.8,51.1 L212.7,51.2 L213.5,51.5 L214.4,51.9 L215.3,52.6 L216.1,53.4 L217.0,54.4 L217.8,55.6
 L218.7,56.9 L219.6,58.4 L220.4,60.2 L221.3,62.1 L222.1,64.2 L223.0,66.4 L223.9,68.9 L224.7,71.6
 L225.6,74.4 L226.4,77.5 L227.3,80.7 L228.2,84.0 L229.0,87.6 L229.9,91.3 L230.7,95.2 L231.6,99.3
 L232.5,103.5 L233.3,107.9 L234.2,112.4 L235.0,117.1 L235.9,121.9 L236.8,126.9 L237.6,132.0 L238.5,137.2
 L239.3,142.6 L240.2,148.1 L241.1,153.7 L241.9,159.5 L242.8,165.3 L243.6,171.2 L244.5,177.3 L245.4,183.4
 L246.2,189.6 L247.1,195.9 L247.9,202.3 L248.8,208.7 L249.7,215.2 L250.5,221.8 L251.4,228.4 L252.2,235.1
 L253.1,241.8 L254.0,248.6 L254.8,255.4 L255.7,262.2 L256.5,269.0 L257.4,275.9 L258.3,282.7 L259.1,289.6
 L260.0,296.4 L260.8,303.3 L261.7,310.1 L262.6,316.9 L263.4,323.7 L264.3,330.5 L265.1,337.2 L266.0,343.9
 L266.9,350.6 L267.7,357.2 L268.6,363.8 L269.4,370.3 L270.3,376.7 L271.1,383.2 L272.0,389.5 L272.9,395.8
 L273.7,402.0 L274.6,408.1 L275.4,414.2 L276.3,420.2 L277.2,426.1 L278.0,431.9 L278.9,437.6

In [ ]:
pltTabE1[2].XrangeMin = 0;
pltTabE1[2].XrangeMax = 4;
var gp = pltTabE1[2].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.1 
 
 
 
 
 3.15 
 
 
 
 
 3.2 
 
 
 
 
 3.25 
 
 
 
 
 3.3 
 
 
 
 
 3.35 
 
 
 
 
 3.4 
 
 
 
 
 3.45 
 
 
 
 
 3.5 
 
 
 
 
 3.55 
 
 
 
 
 3.6 
 
 
 
 
 3.65 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 3.5 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m2Oh01eta07 totalEnergy J686 
 
 
 
 
 m2 O h01 e ta07 totalEnergy J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M377.6,57.1 L431.0,57.1 M62.2,80.2 L63.1,80.1 L63.9,80.0 L64.8,80.1 L65.6,80.2 L66.5,80.5
 L67.4,80.8 L68.2,81.3 L69.1,81.8 L69.9,82.3 L70.8,82.9 L71.7,83.5 L72.5,84.1 L73.4,84.8
 L74.2,85.4 L75.1,86.1 L76.0,86.8 L76.8,87.5 L77.7,88.2 L78.5,88.9 L79.4,89.7 L80.3,90.4
 L81.1,91.2 L82.0,91.9 L82.8,92.7 L83.7,93.5 L84.6,94.2 L85.4,95.0 L86.3,95.8 L87.1,96.5
 L88.0,97.3 L88.9,98.1 L89.7,99.1 L90.6,99.9 L91.4,100.7 L92.3,101.5 L93.2,102.2 L94.0,103.0
 L94.9,103.7 L95.7,104.5 L96.6,105.2 L97.5,106.0 L98.3,106.7 L99.2,107.5 L100.0,108.2 L100.9,109.0
 L101.8,109.7 L102.6,110.5 L103.5,111.3 L104.3,112.0 L105.2,112.8 L106.1,113.6 L106.9,114.3 L107.8,115.1
 L108.6,115.8 L109.5,116.6 L110.4,117.3 L111.2,118.1 L112.1,118.8 L112.9,119.6 L113.8,120.4 L114.7,121.1
 L115.5,121.9 L116.4,122.7 L117.2,123.4 L118.1,124.2 L119.0,125.0 L119.8,125.8 L120.7,126.6 L121.5,127.4
 L122.4,128.1 L123.3,129.0 L124.1,129.8 L125.0,130.6 L125.8,131.4 L126.7,132.2 L127.6,133.1 L128.4,133.9
 L129.3,134.7 L130.1,135.6 L131.0,136.4 L131.8,137.3 L132.7,138.2 L133.6,139.0 L134.4,139.9 L135.3,140.8
 L136.1,141.7 L137.0,142.6 L137.9,143.5 L138.7,144.4 L139.6,145.3 L140.4,146.2 L141.3,147.2 L142.2,148.1
 L143.0,149.1 L143.9,150.0 L144.7,151.0 L145.6,151.9 L146.5,152.8 L147.3,153.8 L148.2,154.8 L149.0,155.8
 L149.9,156.8 L150.8,157.8 L151.6,158.8 L152.5,159.8 L153.3,160.9 L154.2,161.9 L155.1,162.9 L155.9,163.9
 L156.8,165.0 L157.6,166.0 L158.5,167.1 L159.4,168.2 L160.2,169.2 L161.1,170.3 L161.9,171.4 L162.8,172.5
 L163.7,173.5 L164.5,174.7 L165.4,175.8 L166.2,176.9 L167.1,177.9 L168.0,179.1 L168.8,180.2 L169.7,181.4
 L170.5,182.5 L171.4,183.7 L172.3,184.8 L173.1,186.0 L174.0,187.2 L174.8,188.4 L175.7,189.6 L176.6,190.8
 L177.4,192.0 L178.3,193.2 L179.1,194.5 L180.0,195.8 L180.9,197.1 L181.7,198.4 L182.6,199.8 L183.4,201.1
 L184.3,202.5 L185.2,203.8 L186.0,205.2 L186.9,206.7 L187.7,208.1 L188.6,209.6 L189.5,211.1 L190.3,212.6
 L191.2,214.1 L192.0,215.7 L192.9,217.3 L193.8,218.9 L194.6,220.5 L195.5,222.2 L196.3,223.9 L197.2,225.6
 L198.1,227.4 L198.9,229.2 L199.8,231.0 L200.6,232.8 L201.5,234.7 L202.4,236.5 L203.2,238.4 L204.1,240.4
 L204.9,242.4 L205.8,244.3 L206.7,246.3 L207.5,248.4 L208.4,250.5 L209.2,252.6 L210.1,254.8 L211.0,256.9
 L211.8,259.1 L212.7,261.3 L213.5,263.5 L214.4,265.7 L215.3,268.0 L216.1,270.2 L217.0,272.5 L217.8,274.8
 L218.7,277.0 L219.6,279.3 L220.4,281.7 L221.3,284.0 L222.1,286.3 L223.0,288.6 L223.9,290.9 L224.7,293.2
 L225.6,295.5 L226.4,297.8 L227.3,300.1 L228.2,302.4 L229.0,304.6 L229.9,306.9 L230.7,309.1 L231.6,311.3
 L232.5,313.5 L233.3,315.7 L234.2,317.9 L235.0,320.0 L235.9,322.1 L236.8,324.2 L237.6,326.3 L238.5,328.4
 L239.3,330.4 L240.2,332.4 L241.1,334.3 L241.9,336.3 L242.8,338.2 L243.6,340.1 L244.5,341.9 L245.4,343.7
 L246.2,345.5 L247.1,347.2 L247.9,348.9 L248.8,350.6 L249.7,352.2 L250.5,353.8 L251.4,355.4 L252.2,356.9
 L253.1,358.4 L254.0,359.9 L254.8,361.2 L255.7,362.6 L256.5,363.9 L257.4,365.1 L258.3,366.4 L259.1,367.6
 L260.0,368.7 L260.8,369.9 L261.7,371.0 L262.6,372.0 L263.4,373.1 L264.3,374.1 L265.1,375.1 L266.0,376.0
 L266.9,376.9 L267.7,377.8 L268.6,

In [ ]:
// for(int i = 0; i < 4; i++) {
//     Plot2Ddata dat = new Plot2Ddata();
//     var grp = PlotTableDM[i,0].dataGroups[0];
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m2/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

## m=4 cases (eta = {0.7, 0.6, 0.5, 0.3, 0.2})

In [ ]:
Plot2Ddata[] pltTabE3 = new Plot2Ddata[3];
for(int i = 0; i < 3; i++) {
    pltTabE3[i] = new Plot2Ddata();
    PlotTableEnrgy[7,i].dataGroups[0].Format.LineColor = LineColors.Blue;
    pltTabE3[i].AddDataGroup(PlotTableEnrgy[7,i].dataGroups[0]);
    PlotTableEnrgy[8,i].dataGroups[0].Format.LineColor = LineColors.Magenta;
    pltTabE3[i] = pltTabE3[i].Merge(PlotTableEnrgy[8,i]);
    PlotTableEnrgy[9,i].dataGroups[0].Format.LineColor = LineColors.Green;
    pltTabE3[i] = pltTabE3[i].Merge(PlotTableEnrgy[9,i]);
    PlotTableEnrgy[10,i].dataGroups[0].Format.LineColor = LineColors.Orange;
    pltTabE3[i] = pltTabE3[i].Merge(PlotTableEnrgy[10,i]);
    PlotTableEnrgy[10,i].dataGroups[0].Format.LineColor = LineColors.Red;
    pltTabE3[i] = pltTabE3[i].Merge(PlotTableEnrgy[11,i]);
}

In [ ]:
pltTabE3[0].XrangeMin = 0;
pltTabE3[0].XrangeMax = 1.5;
var gp = pltTabE3[0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.2 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m4Oh01eta07 surfaceEnergy J686 
 
 
 
 
 m4 O h01 e ta07 surfaceEnergy J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M355.4,57.1 L408.8,57.1 M53.9,69.8 L56.2,69.8 L58.5,70.9 L60.9,72.7 L63.2,75.3 L65.5,78.4
 L67.8,82.0 L70.1,86.0 L72.5,90.4 L74.8,95.2 L77.1,100.2 L79.4,105.5 L81.7,111.0 L84.1,116.9
 L86.4,123.2 L88.7,129.5 L91.0,136.1 L93.4,143.2 L95.7,150.0 L98.0,157.1 L100.3,164.5 L102.6,172.3
 L105.0,180.0 L107.3,187.8 L109.6,195.6 L111.9,203.3 L114.2,211.5 L116.6,219.0 L118.9,226.9 L121.2,234.8
 L123.5,242.8 L125.8,250.7 L128.2,258.5 L130.5,266.3 L132.8,274.0 L135.1,281.6 L137.4,289.9 L139.8,297.4
 L142.1,304.7 L144.4,311.8 L146.7,318.8 L149.0,326.3 L151.4,332.9 L153.7,339.4 L156.0,346.5 L158.3,352.5
 L160.7,359.1 L163.0,365.1 L165.3,370.5 L167.6,376.8 L169.9,382.3 L172.3,387.5 L174.6,392.5 L176.9,396.9
 L179.2,402.0 L181.5,406.2 L183.9,410.3 L186.2,414.6 L188.5,418.3 L190.8,422.1 L193.1,425.4 L195.5,428.4
 L197.8,431.3 L200.1,434.3 L202.4,436.8 L204.7,439.0 L207.1,440.8 L209.4,442.4 L211.7,443.9 L214.0,445.6
 L216.3,447.0 L218.7,447.6 L221.0,448.3 L223.3,448.9 L225.6,449.4 L228.0,449.8 L230.3,450.0 L232.6,450.2
 L234.9,450.2 L237.2,450.1 L239.6,449.9 L241.9,449.7 L244.2,449.4 L246.5,448.8 L248.8,448.2 L251.2,447.6
 L253.5,446.8 L255.8,445.9 L258.1,444.8 L260.4,443.7 L262.8,442.4 L265.1,441.1 L267.4,439.8 L269.7,438.4
 L272.0,436.8 L274.4,434.9 L276.7,433.3 L279.0,431.5 L281.3,429.6 L283.6,427.6 L286.0,425.7 L288.3,423.9
 L290.6,422.1 L292.9,420.4 L295.2,418.7 L297.6,417.2 L299.9,416.0 L302.2,414.7 L304.5,413.8 L306.9,412.9
 L309.2,412.1 L311.5,411.5 L313.8,411.1 L316.1,410.9 L318.5,410.9 L320.8,411.0 L323.1,411.3 L325.4,411.8
 L327.7,412.4 L330.1,413.2 L332.4,414.1 L334.7,415.6 L337.0,416.8 L339.3,418.0 L341.7,419.2 L344.0,420.5
 L346.3,421.8 L348.6,423.2 L350.9,424.6 L353.3,426.3 L355.6,428.0 L357.9,429.4 L360.2,431.1 L362.5,432.7
 L364.9,434.2 L367.2,435.8 L369.5,437.3 L371.8,438.9 L374.2,440.5 L376.5,442.0 L378.8,443.5 L381.1,444.9
 L383.4,446.3 L385.8,447.7 L388.1,449.0 L390.4,450.4 L392.7,451.7 L395.0,453.2 L397.4,454.4 L399.7,455.7
 L402.0,456.9 L404.3,458.0 L406.6,459.1 L409.0,460.1 L411.3,461.2 L413.6,462.2 L415.9,463.0 L418.2,463.9
 L420.6,464.7 L422.9,465.5 L425.2,466.2 L427.5,466.8 L429.8,467.4 L432.2,467.9 L434.5,468.5 L436.8,468.9
 L439.1,469.3 L441.5,469.7 L443.8,470.0 L446.1,470.1 L448.4,470.1 L450.7,470.5 L453.1,470.5 L455.4,470.8
 L457.7,470.8 L460.0,470.6 L462.3,470.5 L464.7,470.4 L467.0,470.3 L469.3,470.6 L471.6,470.3 L473.9,470.1
 L476.3,469.9 L478.6,470.0 L480.9,469.9 L483.2,469.6 L485.5,469.3 L487.9,469.1 L490.2,468.9 L492.5,468.8
 L494.8,469.0 L497.1,468.9 L499.5,468.8 L501.8,468.5 L504.1,468.4 L506.4,468.3 L508.8,468.2 L511.1,468.1
 L513.4,468.1 L515.7,468.1 L518.0,468.1 L520.4,468.1 L522.7,468.2 L525.0,468.3 L527.3,468.4 L529.6,468.6
 L532.0,468.8 L534.3,469.0 L536.6,469.2 L538.9,469.5 L541.2,470.1 L543.6,470.3 L545.9,470.6 L548.2,471.0
 L550.5,471.3 L552.8,471.7 L555.2,472.1 L557.5,472.5 L559.8,472.9 L562.1,473.4 L564.4,473.9 L566.8,474.3
 L569.1,474.8 L571.4,475.3 L573.7,475.8 L576.0,476.3 L578.4,476.8 L580.7,477.3 L583.0,477.9 L585.3,478.4
 L587.7,478.9 L590.0,479.6 L592.3,480.0 L594.6,480.5 L596.9,481.0 L599.3,481.5 L601.6,481.9 L603.9,482.5
 L606.2,482.9 L608.5,483.3 L610.9,483.7 L613.2,484.1 L615.5,484.6 L617.8,485.0 L620.1,485.4 L62

In [ ]:
pltTabE1[1].XrangeMin = 0;
pltTabE1[1].XrangeMax = 1.5;
var gp = pltTabE1[1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.05 
 
 
 
 
 0.1 
 
 
 
 
 0.15 
 
 
 
 
 0.2 
 
 
 
 
 0.25 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m2Oh01eta07 kineticEnergy J686 
 
 
 
 
 m2 O h01 e ta07 kineticEnergy J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M355.4,57.1 L408.8,57.1 M62.2,564.0 L64.5,563.7 L66.8,562.7 L69.1,561.4 L71.4,559.8 L73.7,558.0
 L76.0,556.1 L78.3,554.1 L80.5,552.0 L82.8,550.0 L85.1,547.8 L87.4,545.7 L89.7,543.5 L92.0,541.3
 L94.3,539.0 L96.6,536.7 L98.9,534.5 L101.2,532.2 L103.5,529.9 L105.8,527.5 L108.1,525.2 L110.4,522.9
 L112.6,520.5 L114.9,518.2 L117.2,515.9 L119.5,513.5 L121.8,511.1 L124.1,508.7 L126.4,506.3 L128.7,503.9
 L131.0,501.5 L133.3,499.1 L135.6,496.6 L137.9,494.1 L140.2,491.6 L142.5,489.1 L144.7,486.5 L147.0,484.0
 L149.3,481.4 L151.6,478.8 L153.9,476.2 L156.2,473.5 L158.5,470.8 L160.8,468.1 L163.1,465.4 L165.4,462.6
 L167.7,459.8 L170.0,457.0 L172.3,454.1 L174.6,451.3 L176.9,448.4 L179.1,445.4 L181.4,442.4 L183.7,439.4
 L186.0,436.4 L188.3,433.3 L190.6,430.3 L192.9,427.1 L195.2,424.0 L197.5,420.8 L199.8,417.6 L202.1,414.3
 L204.4,411.1 L206.7,407.8 L209.0,404.5 L211.2,401.1 L213.5,397.7 L215.8,394.3 L218.1,390.9 L220.4,387.4
 L222.7,384.0 L225.0,380.5 L227.3,376.9 L229.6,373.4 L231.9,369.8 L234.2,366.2 L236.5,362.6 L238.8,359.0
 L241.1,355.4 L243.3,351.7 L245.6,348.0 L247.9,344.3 L250.2,340.6 L252.5,336.9 L254.8,333.2 L257.1,329.4
 L259.4,325.7 L261.7,321.9 L264.0,318.1 L266.3,314.3 L268.6,310.5 L270.9,306.7 L273.2,302.9 L275.4,299.1
 L277.7,295.3 L280.0,291.5 L282.3,287.6 L284.6,283.8 L286.9,280.0 L289.2,276.1 L291.5,272.3 L293.8,268.5
 L296.1,264.7 L298.4,260.8 L300.7,257.0 L303.0,253.2 L305.3,249.3 L307.6,245.5 L309.8,241.7 L312.1,237.9
 L314.4,234.1 L316.7,230.3 L319.0,226.5 L321.3,222.7 L323.6,218.9 L325.9,215.1 L328.2,211.3 L330.5,207.5
 L332.8,203.7 L335.1,200.0 L337.4,196.2 L339.7,192.4 L341.9,188.6 L344.2,184.8 L346.5,181.1 L348.8,177.3
 L351.1,173.5 L353.4,169.8 L355.7,166.0 L358.0,162.3 L360.3,158.6 L362.6,154.8 L364.9,151.1 L367.2,147.4
 L369.5,143.8 L371.8,140.2 L374.0,136.5 L376.3,132.9 L378.6,129.4 L380.9,125.9 L383.2,122.4 L385.5,118.9
 L387.8,115.5 L390.1,112.2 L392.4,108.9 L394.7,105.6 L397.0,102.4 L399.3,99.3 L401.6,96.2 L403.9,93.1
 L406.2,90.2 L408.4,87.3 L410.7,84.5 L413.0,81.8 L415.3,79.2 L417.6,76.6 L419.9,74.2 L422.2,71.9
 L424.5,69.6 L426.8,67.5 L429.1,65.5 L431.4,63.5 L433.7,61.8 L436.0,60.1 L438.3,58.6 L440.5,57.1
 L442.8,55.9 L445.1,54.8 L447.4,53.8 L449.7,52.9 L452.0,52.2 L454.3,51.7 L456.6,51.3 L458.9,51.1
 L461.2,51.1 L463.5,51.2 L465.8,51.5 L468.1,51.9 L470.4,52.6 L472.6,53.4 L474.9,54.4 L477.2,55.6
 L479.5,56.9 L481.8,58.4 L484.1,60.2 L486.4,62.1 L488.7,64.2 L491.0,66.4 L493.3,68.9 L495.6,71.6
 L497.9,74.4 L500.2,77.5 L502.5,80.7 L504.7,84.0 L507.0,87.6 L509.3,91.3 L511.6,95.2 L513.9,99.3
 L516.2,103.5 L518.5,107.9 L520.8,112.4 L523.1,117.1 L525.4,121.9 L527.7,126.9 L530.0,132.0 L532.3,137.2
 L534.6,142.6 L536.9,148.1 L539.1,153.7 L541.4,159.5 L543.7,165.3 L546.0,171.2 L548.3,177.3 L550.6,183.4
 L552.9,189.6 L555.2,195.9 L557.5,202.3 L559.8,208.7 L562.1,215.2 L564.4,221.8 L566.7,228.4 L569.0,235.1
 L571.2,241.8 L573.5,248.6 L575.8,255.4 L578.1,262.2 L580.4,269.0 L582.7,275.9 L585.0,282.7 L587.3,289.6
 L589.6,296.4 L591.9,303.3 L594.2,310.1 L596.5,316.9 L598.8,323.7 L601.1,330.5 L603.3,337.2 L605.6,343.9
 L607.9,350.6 L610.2,357.2 L612.5,363.8 L614.8,370.3 L617.1,376.7 L619.4,383.2 L621.7,389.5 L624.0,395.8
 L626.3,402.0 L628.6,408.1 L630.9,414.2 L633.2,420.2 L635.4,426.1 L637.7

In [ ]:
pltTabE1[2].XrangeMin = 0;
pltTabE1[2].XrangeMax = 1.5;
var gp = pltTabE1[2].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.15 
 
 
 
 
 3.2 
 
 
 
 
 3.25 
 
 
 
 
 3.3 
 
 
 
 
 3.35 
 
 
 
 
 3.4 
 
 
 
 
 3.45 
 
 
 
 
 3.5 
 
 
 
 
 3.55 
 
 
 
 
 3.6 
 
 
 
 
 3.65 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 1.4 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 m2Oh01eta07 totalEnergy J686 
 
 
 
 
 m2 O h01 e ta07 totalEnergy J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M377.6,57.1 L431.0,57.1 M62.2,84.7 L64.5,84.5 L66.8,84.4 L69.1,84.5 L71.4,84.6 L73.7,84.9
 L76.0,85.3 L78.3,85.8 L80.5,86.3 L82.8,86.9 L85.1,87.6 L87.4,88.2 L89.7,88.9 L92.0,89.6
 L94.3,90.3 L96.6,91.1 L98.9,91.9 L101.2,92.6 L103.5,93.4 L105.8,94.2 L108.1,95.0 L110.4,95.9
 L112.6,96.7 L114.9,97.5 L117.2,98.4 L119.5,99.2 L121.8,100.0 L124.1,100.9 L126.4,101.7 L128.7,102.6
 L131.0,103.4 L133.3,104.3 L135.6,105.4 L137.9,106.3 L140.2,107.2 L142.5,108.0 L144.7,108.8 L147.0,109.7
 L149.3,110.5 L151.6,111.3 L153.9,112.1 L156.2,113.0 L158.5,113.8 L160.8,114.6 L163.1,115.5 L165.4,116.3
 L167.7,117.1 L170.0,117.9 L172.3,118.8 L174.6,119.6 L176.9,120.4 L179.1,121.3 L181.4,122.1 L183.7,123.0
 L186.0,123.8 L188.3,124.6 L190.6,125.4 L192.9,126.3 L195.2,127.1 L197.5,127.9 L199.8,128.8 L202.1,129.6
 L204.4,130.5 L206.7,131.3 L209.0,132.2 L211.2,133.0 L213.5,133.9 L215.8,134.7 L218.1,135.6 L220.4,136.5
 L222.7,137.4 L225.0,138.2 L227.3,139.1 L229.6,140.0 L231.9,140.9 L234.2,141.8 L236.5,142.8 L238.8,143.7
 L241.1,144.6 L243.3,145.5 L245.6,146.5 L247.9,147.4 L250.2,148.4 L252.5,149.3 L254.8,150.3 L257.1,151.3
 L259.4,152.3 L261.7,153.2 L264.0,154.2 L266.3,155.2 L268.6,156.2 L270.9,157.3 L273.2,158.3 L275.4,159.3
 L277.7,160.4 L280.0,161.4 L282.3,162.4 L284.6,163.4 L286.9,164.5 L289.2,165.6 L291.5,166.7 L293.8,167.8
 L296.1,168.8 L298.4,170.0 L300.7,171.1 L303.0,172.2 L305.3,173.3 L307.6,174.4 L309.8,175.6 L312.1,176.7
 L314.4,177.9 L316.7,179.0 L319.0,180.2 L321.3,181.4 L323.6,182.6 L325.9,183.7 L328.2,184.9 L330.5,186.1
 L332.8,187.3 L335.1,188.5 L337.4,189.7 L339.7,191.0 L341.9,192.1 L344.2,193.4 L346.5,194.6 L348.8,195.9
 L351.1,197.2 L353.4,198.4 L355.7,199.7 L358.0,201.0 L360.3,202.3 L362.6,203.6 L364.9,204.9 L367.2,206.3
 L369.5,207.6 L371.8,208.9 L374.0,210.3 L376.3,211.8 L378.6,213.2 L380.9,214.7 L383.2,216.1 L385.5,217.6
 L387.8,219.1 L390.1,220.6 L392.4,222.2 L394.7,223.7 L397.0,225.3 L399.3,226.9 L401.6,228.6 L403.9,230.3
 L406.2,231.9 L408.4,233.7 L410.7,235.4 L413.0,237.2 L415.3,239.0 L417.6,240.8 L419.9,242.7 L422.2,244.6
 L424.5,246.5 L426.8,248.5 L429.1,250.5 L431.4,252.5 L433.7,254.5 L436.0,256.6 L438.3,258.7 L440.5,260.8
 L442.8,263.0 L445.1,265.1 L447.4,267.4 L449.7,269.6 L452.0,272.0 L454.3,274.3 L456.6,276.6 L458.9,279.0
 L461.2,281.4 L463.5,283.8 L465.8,286.2 L468.1,288.7 L470.4,291.2 L472.6,293.6 L474.9,296.1 L477.2,298.6
 L479.5,301.1 L481.8,303.7 L484.1,306.2 L486.4,308.8 L488.7,311.3 L491.0,313.9 L493.3,316.4 L495.6,318.9
 L497.9,321.4 L500.2,324.0 L502.5,326.5 L504.7,329.0 L507.0,331.5 L509.3,334.0 L511.6,336.4 L513.9,338.9
 L516.2,341.3 L518.5,343.7 L520.8,346.1 L523.1,348.4 L525.4,350.8 L527.7,353.0 L530.0,355.3 L532.3,357.6
 L534.6,359.8 L536.9,362.0 L539.1,364.2 L541.4,366.3 L543.7,368.4 L546.0,370.5 L548.3,372.5 L550.6,374.5
 L552.9,376.4 L555.2,378.4 L557.5,380.2 L559.8,382.1 L562.1,383.8 L564.4,385.6 L566.7,387.3 L569.0,389.0
 L571.2,390.6 L573.5,392.2 L575.8,393.7 L578.1,395.2 L580.4,396.7 L582.7,398.0 L585.0,399.4 L587.3,400.7
 L589.6,402.0 L591.9,403.3 L594.2,404.5 L596.5,405.6 L598.8,406.7 L601.1,407.9 L603.3,409.0 L605.6,410.0
 L607.9,411.0 L610.2,412.0 L612

In [ ]:
for(int i = 0; i < 4; i++) {
    Plot2Ddata dat = new Plot2Ddata();
    var grp = PlotTableDM[i,0].dataGroups[0];
    dat.AddDataGroup(grp);
    string name = "plotData/AspectRatioOverTime/m2/" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}